In [1]:
import re
import json
import csv
from io import StringIO
from bs4 import BeautifulSoup
import requests

In [2]:
# url templates
url_stats = 'https://finance.yahoo.com/quote/{}/key-statistics?p={}'
url_profile = 'https://finance.yahoo.com/quote/{}/profile?p={}'
url_financials = 'https://finance.yahoo.com/quote/{}/financials?p={}'
url_financials_balance = 'https://finance.yahoo.com/quote/{}/balance-sheet?p={}'
url_financials_cashflow = 'https://finance.yahoo.com/quote/{}/cash-flow?p={}'

# the stock I want to scrape
stock = 'TSLA'

In [3]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36"
}

In [4]:
response = requests.get(url_profile.format(stock, stock),headers=headers)
print(response.status_code)
soup = BeautifulSoup(response.text, 'html.parser')

table = soup.find('table', {'class': 'yf-mj92za'})

headers = [header.get_text(strip=True) for header in table.find_all('th')]

rows=[]

for row in table.find_all('tr')[1:]:
    columns = row.find_all('td')
    
    if len(columns)>0:
        data ={
            headers[0]: columns[0].get_text(strip=True),
            headers[1]: columns[1].get_text(strip=True),
            headers[2]: columns[2].get_text(strip=True),
            headers[3]: columns[3].get_text(strip=True),
            headers[4]: columns[4].get_text(strip=True)
        }
        rows.append(data)

200


In [5]:
for row in rows:
    print(row)

{'Name': 'Mr. Elon R. Musk', 'Title': 'Co-Founder, Technoking of Tesla, CEO & Director', 'Pay': '--', 'Exercised': '--', 'Year Born': '1972'}
{'Name': 'Mr. Vaibhav  Taneja', 'Title': 'Chief Financial Officer', 'Pay': '278k', 'Exercised': '8.52M', 'Year Born': '1978'}
{'Name': 'Mr. Xiaotong  Zhu', 'Title': 'Senior Vice President of Automotive', 'Pay': '926.88k', 'Exercised': '--', 'Year Born': '1980'}
{'Name': 'Travis  Axelrod', 'Title': 'Head of Investor Relations', 'Pay': '--', 'Exercised': '--', 'Year Born': '--'}
{'Name': 'Brian  Scelfo', 'Title': 'Senior Director of Corporate Development', 'Pay': '--', 'Exercised': '--', 'Year Born': '--'}
{'Name': 'Mr. Franz  von Holzhausen', 'Title': 'Chief Designer', 'Pay': '--', 'Exercised': '--', 'Year Born': '--'}
{'Name': 'Mr. John  Walker', 'Title': 'Vice President of Sales - North America', 'Pay': '121.55k', 'Exercised': '--', 'Year Born': '1963'}
{'Name': 'Mr. Peter  Bannon', 'Title': 'Chip Architect', 'Pay': '--', 'Exercised': '--', 'Yea

Finance Income sheet

In [6]:
response_finance = requests.get(url_financials.format(stock, stock),headers=headers)
print(response_finance.status_code)
soup = BeautifulSoup(response_finance.text, 'html.parser')
table = soup.find('div', {'class': 'tableBody yf-9ft13'})

row = soup.find('div', {'class': 'row yf-1yyu1pc'})
header = [div.get_text(strip=True) for div in row.find_all('div')]

data_finance = []
for row in table.find_all('div', {'class': 'row lv-0 yf-t22klz'}):
    row_data = {}
    columns1 = row.find_all('div', {'class': 'column yf-t22klz'})

    columns2 = row.find_all('div', {'class': 'column yf-t22klz alt'})

    columns = columns1 + columns2
    
    # Extract the text for each row
    row_data[header[0]] = row.find('div', {'class': 'rowTitle yf-t22klz'}).get_text(strip=True)
    for i, column in enumerate(columns):
        row_data[header[i+1]] = column.get_text(strip=True)
    
    data_finance.append(row_data)

AttributeError: 'list' object has no attribute 'items'

In [5]:
for entry in data_finance:
    print(entry)

{'Breakdown': 'Total Revenue', 'TTM': '176,191,000.00', '12/31/2023': '136,341,000.00', '12/31/2022': '182,743,000.00', '12/31/2021': '158,057,000.00', '12/31/2020': '127,144,000.00'}
{'Breakdown': 'Cost of Revenue', 'TTM': '160,031,000.00', '12/31/2023': '119,903,000.00', '12/31/2022': '168,715,000.00', '12/31/2021': '140,893,000.00', '12/31/2020': '121,359,000.00'}
{'Breakdown': 'Gross Profit', 'TTM': '16,160,000.00', '12/31/2023': '16,438,000.00', '12/31/2022': '14,028,000.00', '12/31/2021': '17,164,000.00', '12/31/2020': '5,785,000.00'}
{'Breakdown': 'Operating Expense', 'TTM': '10,702,000.00', '12/31/2023': '11,915,000.00', '12/31/2022': '10,285,000.00', '12/31/2021': '10,888,000.00', '12/31/2020': '10,193,000.00'}
{'Breakdown': 'Operating Income', 'TTM': '5,458,000.00', '12/31/2023': '4,523,000.00', '12/31/2022': '3,743,000.00', '12/31/2021': '6,276,000.00', '12/31/2020': '-4,408,000.00'}
{'Breakdown': 'Net Non Operating Interest Income Expense', 'TTM': '249,000.00', '12/31/2023'

In [8]:
output_file = "financial_data.json"
with open(output_file, "w") as json_file:
    json.dump(data_finance, json_file, indent=4)

print(f"Data has been successfully saved to {output_file}")

Data has been successfully saved to financial_data.json


In [9]:
output_file = "financial_data.csv"

# Writing to CSV
with open(output_file, mode='w', newline='', encoding='utf-8') as csv_file:
    writer = csv.DictWriter(csv_file, fieldnames=data_finance[0].keys())
    writer.writeheader()  # Write the header row
    writer.writerows(data_finance)  # Write the data rows

print(f"Data has been successfully saved to {output_file}")

Data has been successfully saved to financial_data.csv


Finance Balance sheet

In [7]:
print(url_financials_balance.format(stock, stock))

https://finance.yahoo.com/quote/F/balance-sheet?p=F


In [12]:
response_balance = requests.get(url_financials_balance.format(stock, stock),headers=headers)
print(response_balance.status_code)
soup = BeautifulSoup(response_balance.text, 'html.parser')
table = soup.find('div', {'class': 'tableBody yf-9ft13'})

row = soup.find('div', {'class': 'row yf-1yyu1pc'})
header = [div.get_text(strip=True) for div in row.find_all('div')]

data_balance = []
for row in table.find_all('div', {'class': 'row lv-0 yf-t22klz'}):
    row_data = {}
    columns1 = row.find_all('div', {'class': 'column yf-t22klz'})

    columns2 = row.find_all('div', {'class': 'column yf-t22klz alt'})

    columns = columns1 + columns2
    
    # Extract the text for each row
    row_data[header[0]] = row.find('div', {'class': 'rowTitle yf-t22klz'}).get_text(strip=True)
    for i, column in enumerate(columns):
        row_data[header[i+1]] = column.get_text(strip=True)
    
    data_balance.append(row_data)

200


In [13]:
for row in data_balance:
    print(row)

{'Breakdown': 'Total Assets', '12/31/2023': '255,884,000.00', '12/31/2022': '267,261,000.00', '12/31/2021': '273,310,000.00', '12/31/2020': '257,035,000.00'}
{'Breakdown': 'Total Liabilities Net Minority Interest', '12/31/2023': '212,717,000.00', '12/31/2022': '236,450,000.00', '12/31/2021': '230,512,000.00', '12/31/2020': '208,413,000.00'}
{'Breakdown': 'Total Equity Gross Minority Interest', '12/31/2023': '43,167,000.00', '12/31/2022': '30,811,000.00', '12/31/2021': '42,798,000.00', '12/31/2020': '48,622,000.00'}
{'Breakdown': 'Total Capitalization', '12/31/2023': '132,047,000.00', '12/31/2022': '141,031,000.00', '12/31/2021': '142,335,000.00', '12/31/2020': '136,919,000.00'}
{'Breakdown': 'Common Stock Equity', '12/31/2023': '43,242,000.00', '12/31/2022': '30,690,000.00', '12/31/2021': '42,773,000.00', '12/31/2020': '48,519,000.00'}
{'Breakdown': 'Capital Lease Obligations', '12/31/2023': '1,505,000.00', '12/31/2022': '1,314,000.00', '12/31/2021': '1,876,000.00', '12/31/2020': '1,39

In [14]:
output_file = "financial_balance_data.json"
with open(output_file, "w") as json_file:
    json.dump(data_balance, json_file, indent=4)

print(f"Data has been successfully saved to {output_file}")

Data has been successfully saved to financial_balance_data.json


In [22]:
output_file = "financial_balance_data.csv"

# Writing to CSV
with open(output_file, mode='w', newline='', encoding='utf-8') as csv_file:
    writer = csv.DictWriter(csv_file, fieldnames=data_balance[0].keys())
    writer.writeheader()  # Write the header row
    writer.writerows(data_balance)  # Write the data rows

print(f"Data has been successfully saved to {output_file}")

Data has been successfully saved to financial_balance_data.csv


Finance cash-flow

In [16]:
response_cashflow = requests.get(url_financials_cashflow.format(stock, stock),headers=headers)
print(response_cashflow.status_code)
soup = BeautifulSoup(response_cashflow.text, 'html.parser')
table = soup.find('div', {'class': 'tableBody yf-9ft13'})

row = soup.find('div', {'class': 'row yf-1yyu1pc'})
header = [div.get_text(strip=True) for div in row.find_all('div')]

data_cashflow = []
for row in table.find_all('div', {'class': 'row lv-0 yf-t22klz'}):
    row_data = {}
    columns1 = row.find_all('div', {'class': 'column yf-t22klz'})

    columns2 = row.find_all('div', {'class': 'column yf-t22klz alt'})

    columns = columns1 + columns2
    
    # Extract the text for each row
    row_data[header[0]] = row.find('div', {'class': 'rowTitle yf-t22klz'}).get_text(strip=True)
    for i, column in enumerate(columns):
        row_data[header[i+1]] = column.get_text(strip=True)
    
    data_cashflow.append(row_data)

200


In [18]:
output_file = "financial_cashflow_data.json"
with open(output_file, "w") as json_file:
    json.dump(data_cashflow, json_file, indent=4)

print(f"Data has been successfully saved to {output_file}")

Data has been successfully saved to financial_cashflow_data.json


In [23]:
output_file = "financial_cashflow_data.csv"

# Writing to CSV
with open(output_file, mode='w', newline='', encoding='utf-8') as csv_file:
    writer = csv.DictWriter(csv_file, fieldnames=data_cashflow[0].keys())
    writer.writeheader()  # Write the header row
    writer.writerows(data_cashflow)  # Write the data rows

print(f"Data has been successfully saved to {output_file}")

Data has been successfully saved to financial_cashflow_data.csv


In [21]:
for row in data_cashflow:
    print(row)

{'Breakdown': 'Operating Cash Flow', 'TTM': '14,918,000.00', '12/31/2023': '15,787,000.00', '12/31/2022': '14,887,000.00', '12/31/2021': '6,853,000.00', '12/31/2020': '24,269,000.00'}
{'Breakdown': 'Investing Cash Flow', 'TTM': '-17,628,000.00', '12/31/2023': '2,745,000.00', '12/31/2022': '-24,918,000.00', '12/31/2021': '-4,347,000.00', '12/31/2020': '-18,615,000.00'}
{'Breakdown': 'Financing Cash Flow', 'TTM': '2,584,000.00', '12/31/2023': '-23,498,000.00', '12/31/2022': '7,078,000.00', '12/31/2021': '2,511,000.00', '12/31/2020': '2,315,000.00'}
{'Breakdown': 'End Cash Position', 'TTM': '25,110,000.00', '12/31/2023': '20,737,000.00', '12/31/2022': '23,681,000.00', '12/31/2021': '25,340,000.00', '12/31/2020': '25,935,000.00'}
{'Breakdown': 'Capital Expenditure', 'TTM': '-8,236,000.00', '12/31/2023': '-6,227,000.00', '12/31/2022': '-8,481,000.00', '12/31/2021': '-6,866,000.00', '12/31/2020': '-5,742,000.00'}
{'Breakdown': 'Issuance of Debt', 'TTM': '51,659,000.00', '12/31/2023': '27,901

In [ ]:
response_statistics = requests.get(url_stats.format(stock, stock),headers=headers)
print(response_statistics.status_code)
soup = BeautifulSoup(response_statistics.text, 'html.parser')

table = soup.find('table', {'class': 'table yf-kbx2lo'})

headers = [header.get_text(strip=True) for header in table.find_all('th')]

stat_data=[]

for row in table.find_all('tr')[1:]:
    columns = row.find_all('td')
    
    if len(columns)>0:
        data ={
            "Name": columns[0].get_text(strip=True),
            headers[1]: columns[1].get_text(strip=True),
            headers[2]: columns[2].get_text(strip=True),
            headers[3]: columns[3].get_text(strip=True),
            headers[4]: columns[4].get_text(strip=True)
        }
        stat_data.append(data)

200


In [5]:
for row in stat_data:
    print(row)

{'Name': 'Market Cap', 'Current': '39.35B', '9/30/2024': '41.98B', '6/30/2024': '50.06B', '3/31/2024': '52.77B'}
{'Name': 'Enterprise Value', 'Current': '161.46B', '9/30/2024': '160.31B', '6/30/2024': '167.02B', '3/31/2024': '163.71B'}
{'Name': 'Trailing P/E', 'Current': '10.97', '9/30/2024': '11.00', '6/30/2024': '12.93', '3/31/2024': '12.30'}
{'Name': 'Forward P/E', 'Current': '5.48', '9/30/2024': '5.33', '6/30/2024': '6.06', '3/31/2024': '7.22'}
{'Name': 'PEG Ratio (5yr expected)', 'Current': '0.55', '9/30/2024': '0.57', '6/30/2024': '0.66', '3/31/2024': '0.78'}
{'Name': 'Price/Sales', 'Current': '0.21', '9/30/2024': '0.24', '6/30/2024': '0.29', '3/31/2024': '0.30'}
{'Name': 'Price/Book', 'Current': '0.87', '9/30/2024': '0.96', '6/30/2024': '1.17', '3/31/2024': '1.23'}
{'Name': 'Enterprise Value/Revenue', 'Current': '0.88', '9/30/2024': '0.89', '6/30/2024': '0.94', '3/31/2024': '0.93'}
{'Name': 'Enterprise Value/EBITDA', 'Current': '15.46', '9/30/2024': '14.36', '6/30/2024': '15.02'